# Local search

局所検索法は、知識グラフの構造化データと入力文書の非構造化データを組み合わせ、クエリ時に関連するエンティティ情報でLLMコンテキストを補強する。これは、入力文書で言及されている特定のエンティティの理解を必要とする質問に答えるのに適している（例えば、「カモミールの治癒特性は何か？）

In [1]:
%load_ext dotenv

import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

ベースとなるLLMとEmbeddingの指定

In [2]:
api_key = os.environ["OPENAI_API_KEY"]
llm_model = 'gpt-4o'
embedding_model = 'text-embedding-3-small'

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

`python -m graphrag.index --root ./ragtest`で生成された`.parquet`ファイルとlanceDBへのpathを指定

In [3]:
input_dir = "data/graphrag/gs_q6OLkTkx9NIQ0Wobtqn62tOy/output/default/artifacts"
lancedb_uri = f"{input_dir}/lancedb"

community_report_table = "create_final_community_reports"
entity_table = "create_final_nodes"
entity_embedding_table = "create_final_entities"
relationship_table = "create_final_relationships"
covariate_table = "create_final_covariates"
text_unit_table = "create_final_text_units"

`pd.DataFrame`として読み出し

In [4]:
report_df = pd.read_parquet(f"{input_dir}/{community_report_table}.parquet")
entity_df = pd.read_parquet(f"{input_dir}/{entity_table}.parquet")
entity_embedding_df = pd.read_parquet(f"{input_dir}/{entity_embedding_table}.parquet")
relationship_df = pd.read_parquet(f"{input_dir}/{relationship_table}.parquet")
covariate_df = pd.read_parquet(f"{input_dir}/{covariate_table}.parquet")
text_unit_df = pd.read_parquet(f"{input_dir}/{text_unit_table}.parquet")

DataFrameからGraphRAGとして使える形に変換

In [5]:
reports = read_indexer_reports(report_df, entity_df, 2)
entities = read_indexer_entities(entity_df, entity_embedding_df, 2)
relationships = read_indexer_relationships(relationship_df)
claims = read_indexer_covariates(covariate_df)
covariates = {"claims": claims}
text_units = read_indexer_text_units(text_unit_df)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=lancedb_uri)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

[2024-08-12T06:14:15Z WARN  lance::dataset] No existing dataset at /Users/hashyuki/code/learn_graphrag/data/output/test/artifacts/lancedb/entity_description_embeddings.lance, it will be created


ローカルな文脈を構築

In [6]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

ローカルサーチのためのエンジンをインスタンス化

In [7]:
local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [8]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

ローカルサーチ

In [9]:
result = await search_engine.asearch(
    "主人公の交友関係を説明して"
)

print(result.response)

# 主人公の交友関係

## 清峰葉流火と要圭の関係

清峰葉流火と要圭は中学時代に天才バッテリーとして恐れられていたコンビです。彼らの強力なパートナーシップは、野球界で多くの注目を集めました。しかし、要圭が記憶喪失になり、野球の素人となったことで、彼らの関係は新たな局面を迎えます。要圭が小手指高校に入学し、清峰もそれに続いて入学することで、再び彼らの友情とパートナーシップが試されることになります [Data: Entities (112, 113); Relationships (11, 14)]。

## 藤堂葵と千早瞬平の関係

藤堂葵と千早瞬平もまた、清峰葉流火と要圭に影響を受けたキャラクターです。彼らは中学時代に清峰と要に敗北し、野球を辞める決意をしました。しかし、運命のいたずらか、小手指高校で再び出会い、共に野球部に入部することになります。藤堂と千早の友情は、彼らが再び野球に挑戦するための大きな支えとなります [Data: Entities (114, 115); Relationships (17, 18, 19)]。

## 山田太郎の関係

山田太郎は、清峰葉流火と要圭に対する強い影響を受けたもう一人のキャラクターです。彼は中学時代に彼らと対戦し、その結果として野球を辞める決意をしました。しかし、小手指高校に進学した後、再び清峰と要に出会い、彼の人生は大きく変わります。山田の物語は、過去の挫折を乗り越え、新たな挑戦に立ち向かう姿を描いています [Data: Entities (113, 116); Relationships (12, 16)]。

## 野球部の仲間たち

小手指高校の野球部は、これらのキャラクターたちが再び集まり、共に新たな挑戦に立ち向かう場です。清峰葉流火、要圭、藤堂葵、千早瞬平、そして山田太郎が一緒に野球部に入部し、彼らの友情とチームワークが試されます。野球部は、彼らが過去の挫折を乗り越え、再び野球に情熱を注ぐための重要な舞台となります [Data: Entities (117); Relationships (14, 15, 18, 19)]。

## 結論

主人公たちの交友関係は、彼らの過去の経験と現在の挑戦が交錯する複雑なものです。清峰葉流火と要圭の強力なパートナーシップ、藤堂葵と千早瞬平の友情、そして山田太郎の再起

In [10]:
result.context_data["reports"]

,id,title,content
0,3,Mura and the Dragon's Legacy,# Mura and the Dragon's Legacy\n\nThe communit...


In [11]:
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 1. LLM tokens: 5982
